# Show results of a specific model

In [ ]:
MODEL_ID = "2022_03_27-20_51_13"
WIDTH = 8
HEIGHT = 8

In [ ]:
%cd ../..

In [ ]:
import os
import pandas as pd

from scripts import config
from scripts.tools.get_name import get_model_img_name
import matplotlib.pyplot as plt

In [ ]:
def show_results(model_id):
    df_runs = pd.read_csv(config.REMOTE_PARAMS_PATH, index_col=0)
    df_result = df_runs.loc[df_runs['run_id'] == model_id]
    run = df_result.iloc[0]

    pictures_path = os.path.join(config.LOGS_FOLDER, run["run_id"], "pictures")
    epoch_pictures_path = os.path.join(
        pictures_path, f"epoch_{run['epochs_trained_nb']}"
    )
    print(
        f"""Model {run['run_id']} with epoch {run['epochs_trained_nb']}:
    - Architecture: {run['cartoon_gan_architecture']}
    - Pictures size: {run['picture_dataset_new_size']}
    - Learning rate: {run['training_gen_lr']}
    - Crop mode: {run['picture_dataset_crop_mode']}
    - Init generator path: {run['init_gen_path']}

    With losses:
    - Generator loss: {run['train_gen_loss']}
    - Discriminator loss: {run['train_disc_loss']}"""
    )

    columns = 2
    nb_pictures = len(os.listdir(epoch_pictures_path))//2
    fig = plt.figure(figsize=(WIDTH*columns, HEIGHT*nb_pictures))
    for i in range(nb_pictures):
        picture = plt.imread(os.path.join(epoch_pictures_path, get_model_img_name(i, "picture")))
        cartoon = plt.imread(os.path.join(epoch_pictures_path, get_model_img_name(i, "cartoon")))
        ax = fig.add_subplot(nb_pictures, columns, 2*i+1)
        ax.imshow(picture)
        ax.axis("off")
        ax = fig.add_subplot(nb_pictures, columns, 2*i+2)
        ax.imshow(cartoon)
        ax.axis("off")

In [ ]:
show_results(MODEL_ID)